<a href="https://colab.research.google.com/github/odeandialamsyah/nlp_zero_to_hero/blob/main/NLP_Zero_to_Hero_Pemodelan_Topik.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Untuk Pemodelan Topik, kita akan fokus pada algoritma Latent Dirichlet Allocation (LDA), karena ini adalah yang paling umum dan kuat untuk pengenalan topik. Kita akan menggunakan library Gensim lagi.

Persiapan Data untuk LDA

LDA memerlukan input dalam bentuk daftar dokumen yang sudah di-tokenisasi dan biasanya juga sudah pra-proses (lower-case, tanpa stopwords, dll.).

In [1]:
pip install gensim

In [2]:
# Import library yang dibutuhkan
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

In [3]:
# Pastikan data NLTK sudah diunduh
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

# 1. Contoh Dokumen Mentah
dokumen_mentah = [
    "Saya suka makan pizza dan pasta. Makanan Italia sangat lezat.",
    "Harga saham perusahaan teknologi naik. Investor senang dengan pasar.",
    "Presiden bertemu dengan menteri untuk membahas kebijakan baru. Politik sedang hangat.",
    "Anjing dan kucing adalah hewan peliharaan yang lucu. Mereka suka bermain di rumah.",
    "Saya menikmati pizza dan anjing saya. Apel juga sehat."
]

print("Dokumen Mentah:")
for i, doc in enumerate(dokumen_mentah):
    print(f"Dokumen {i+1}: {doc}")
print("-" * 50)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Dokumen Mentah:
Dokumen 1: Saya suka makan pizza dan pasta. Makanan Italia sangat lezat.
Dokumen 2: Harga saham perusahaan teknologi naik. Investor senang dengan pasar.
Dokumen 3: Presiden bertemu dengan menteri untuk membahas kebijakan baru. Politik sedang hangat.
Dokumen 4: Anjing dan kucing adalah hewan peliharaan yang lucu. Mereka suka bermain di rumah.
Dokumen 5: Saya menikmati pizza dan anjing saya. Apel juga sehat.
--------------------------------------------------


In [4]:
# 2. Pra-pemrosesan Teks (Mengulang langkah yang sudah kita pelajari)
def preprocess_text(text):
    text = text.lower()
    # Menghilangkan tanda baca
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    tokens = word_tokenize(text)

    # Filter stopwords (gunakan stop_words bahasa Indonesia jika tersedia atau English)
    # NLTK memiliki 'indonesian' stopwords, tapi untuk contoh ini saya pakai English
    # karena data contoh campur & lebih ilustratif untuk WordNetLemmatizer yang English-centric.
    # Untuk data real Indonesia, pastikan pakai list stopwords Bahasa Indonesia.
    list_stopwords = set(stopwords.words('indonesian')) # Menggunakan stopwords Indonesia

    # Lemmatization (lebih cocok untuk B.Inggris di NLTK, kita lewati untuk B.Indonesia murni)
    # Untuk demo ini, kita asumsikan bahasa adalah Inggris untuk fungsi lemmatizer NLTK
    # Jika teks Anda murni B.Indonesia, langkah lemmatisasi NLTK ini kurang relevan
    # dan Anda perlu library seperti Sastrawi untuk stemming/lemmatisasi B.Indonesia.
    # Untuk contoh ini, mari kita asumsikan teks ini sudah "dilakukan" stemming/lemmatisasi
    # atau kita lewati WordNetLemmatizer untuk menghindari kebingungan jika teks campur.

    # Kita akan melakukan filtering stop words saja untuk contoh B.Indonesia ini
    filtered_tokens = [word for word in tokens if word not in list_stopwords]
    return filtered_tokens

dokumen_bersih_tokenized = [preprocess_text(doc) for doc in dokumen_mentah]

print("Dokumen Bersih (Tokenized):")
for i, doc in enumerate(dokumen_bersih_tokenized):
    print(f"Dokumen {i+1}: {doc}")
print("-" * 50)

Dokumen Bersih (Tokenized):
Dokumen 1: ['suka', 'makan', 'pizza', 'pasta', 'makanan', 'italia', 'lezat']
Dokumen 2: ['harga', 'saham', 'perusahaan', 'teknologi', 'investor', 'senang', 'pasar']
Dokumen 3: ['presiden', 'bertemu', 'menteri', 'membahas', 'kebijakan', 'politik', 'hangat']
Dokumen 4: ['anjing', 'kucing', 'hewan', 'peliharaan', 'lucu', 'suka', 'bermain', 'rumah']
Dokumen 5: ['menikmati', 'pizza', 'anjing', 'apel', 'sehat']
--------------------------------------------------


In [5]:
# 3. Membuat Kamus (Dictionary) dari Dokumen yang Sudah di-Tokenisasi
# Gensim Dictionary memetakan setiap kata unik ke ID numerik
dictionary = corpora.Dictionary(dokumen_bersih_tokenized)
print("Kamus (Word to ID Mapping):")
# Tampilkan beberapa item dari kamus
print(list(dictionary.items())[:10]) # Tampilkan 10 item pertama
print(f"Total kata unik dalam kamus: {len(dictionary)}\n")

Kamus (Word to ID Mapping):
[(0, 'italia'), (1, 'lezat'), (2, 'makan'), (3, 'makanan'), (4, 'pasta'), (5, 'pizza'), (6, 'suka'), (7, 'harga'), (8, 'investor'), (9, 'pasar')]
Total kata unik dalam kamus: 31



In [6]:
# 4. Membuat Korpus (Bag-of-Words Representasi)
# Mengubah setiap dokumen menjadi representasi Bag-of-Words (frekuensi kata)
# Ini adalah list of lists of (word_id, word_count) tuples
corpus = [dictionary.doc2bow(doc) for doc in dokumen_bersih_tokenized]
print("Representasi Korpus (Bag-of-Words untuk LDA):")
# Tampilkan representasi BoW untuk dokumen pertama sebagai contoh
print(f"Dokumen 1 BoW: {corpus[0]}")
# Untuk memahami: [(id_kata_pizza, 1), (id_kata_pasta, 1), ...]
print("-" * 50)

Representasi Korpus (Bag-of-Words untuk LDA):
Dokumen 1 BoW: [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1)]
--------------------------------------------------


In [7]:
# 5. Melatih Model LDA
# num_topics: Jumlah topik yang ingin kita identifikasi
# id2word: Kamus yang memetakan ID kembali ke kata
# passes: Berapa kali algoritma melewati seluruh korpus
# random_state: Untuk memastikan hasil yang bisa direproduksi
num_topics = 3 # Kita ingin mencari 3 topik
lda_model = LdaModel(corpus=corpus,
                     id2word=dictionary,
                     num_topics=num_topics,
                     passes=15, # Jumlah iterasi pelatihan
                     random_state=100)

print("\n--- Hasil Pemodelan Topik (LDA) ---")


--- Hasil Pemodelan Topik (LDA) ---


In [8]:
# 6. Menampilkan Topik yang Ditemukan
# lda_model.print_topics() akan menampilkan kata-kata kunci untuk setiap topik
print("Kata-kata kunci untuk setiap Topik:")
for idx, topic in lda_model.print_topics(num_words=5): # Tampilkan 5 kata teratas
    print(f"Topik {idx+1}: {topic}")
print("-" * 50)

Kata-kata kunci untuk setiap Topik:
Topik 1: 0.055*"senang" + 0.055*"teknologi" + 0.055*"harga" + 0.055*"presiden" + 0.055*"hangat"
Topik 2: 0.077*"pizza" + 0.077*"suka" + 0.077*"italia" + 0.077*"pasta" + 0.077*"lezat"
Topik 3: 0.100*"anjing" + 0.057*"bermain" + 0.057*"peliharaan" + 0.057*"hewan" + 0.057*"lucu"
--------------------------------------------------


In [9]:
# 7. Menentukan Distribusi Topik untuk Setiap Dokumen
print("Distribusi Topik per Dokumen:")
for i, doc_bow in enumerate(corpus):
    # get_document_topics() akan memberikan distribusi topik untuk dokumen tersebut
    doc_topics = lda_model.get_document_topics(doc_bow)
    # Urutkan berdasarkan bobot topik tertinggi untuk mudah dibaca
    sorted_topics = sorted(doc_topics, key=lambda x: x[1], reverse=True)

    original_doc = dokumen_mentah[i]
    print(f"Dokumen {i+1}: '{original_doc}'")
    print(f"   Topik: {[(f'Topik {topic_id+1}', f'{prob:.2f}') for topic_id, prob in sorted_topics]}")
print("-" * 50)

Distribusi Topik per Dokumen:
Dokumen 1: 'Saya suka makan pizza dan pasta. Makanan Italia sangat lezat.'
   Topik: [('Topik 2', '0.91'), ('Topik 3', '0.04'), ('Topik 1', '0.04')]
Dokumen 2: 'Harga saham perusahaan teknologi naik. Investor senang dengan pasar.'
   Topik: [('Topik 1', '0.92'), ('Topik 2', '0.04'), ('Topik 3', '0.04')]
Dokumen 3: 'Presiden bertemu dengan menteri untuk membahas kebijakan baru. Politik sedang hangat.'
   Topik: [('Topik 1', '0.92'), ('Topik 2', '0.04'), ('Topik 3', '0.04')]
Dokumen 4: 'Anjing dan kucing adalah hewan peliharaan yang lucu. Mereka suka bermain di rumah.'
   Topik: [('Topik 3', '0.92'), ('Topik 2', '0.04'), ('Topik 1', '0.04')]
Dokumen 5: 'Saya menikmati pizza dan anjing saya. Apel juga sehat.'
   Topik: [('Topik 3', '0.89'), ('Topik 2', '0.06'), ('Topik 1', '0.06')]
--------------------------------------------------


Penjelasan Kode Pemodelan Topik (LDA):
1. Pra-pemrosesan (ulang): Kita mengulang langkah pra-pemrosesan yang sudah Anda kuasai. Penting untuk memiliki teks yang bersih dan di-tokenisasi sebelum membangun model topik. Di sini, kita menggunakan stopwords Bahasa Indonesia. Perlu dicatat bahwa WordNetLemmatizer dari NLTK paling efektif untuk Bahasa Inggris. Jika korpus Anda murni Bahasa Indonesia, Anda akan memerlukan library stemming/lemmatization Bahasa Indonesia seperti Sastrawi atau InStem. Untuk contoh di atas, saya hanya menerapkan lowercase dan penghilangan stopwords (Indonesia) saja untuk kesederhanaan.

2. dictionary=corpora.Dictionary(dokumen_bersih_tokenized): Ini adalah langkah penting Gensim. Ia memindai semua dokumen yang sudah di-tokenisasi dan membangun sebuah kamus (vocabulary). Setiap kata unik di korpus diberi ID numerik.

3. corpus = [dictionary.doc2bow(doc) for doc in dokumen_bersih_tokenized]: Setelah kamus dibuat, setiap dokumen diubah menjadi representasi Bag-of-Words (BoW) dalam format Gensim. Ini adalah daftar tuple (word_id, word_count). LDA menerima input dalam format ini.

4. lda_model = LdaModel(...): Inisialisasi dan melatih model LDA.

 - corpus: Data BoW yang kita buat.

 - id2word: Kamus yang dibuat sebelumnya. Ini digunakan model untuk memetakan ID kembali ke kata saat menampilkan topik.

 - num_topics: Ini adalah parameter krusial. Anda perlu menentukan berapa banyak topik yang ingin Anda "temukan". Pilihan ini seringkali berdasarkan domain pengetahuan atau melalui evaluasi performa model.

 - passes: Jumlah kali algoritma LDA mengiterasi seluruh korpus. Semakin banyak passes, semakin baik model belajar, tapi semakin lama.

 - random_state: Untuk memastikan hasil yang sama setiap kali Anda menjalankan kode.

6. lda_model.print_topics(num_words=5): Setelah model dilatih, Anda bisa melihat topik-topik yang ditemukan. Setiap topik direpresentasikan oleh daftar kata-kata yang paling mungkin muncul dalam topik tersebut, bersama dengan bobotnya. Ini membantu Anda menginterpretasikan topik.

7. lda_model.get_document_topics(doc_bow): Untuk setiap dokumen, Anda bisa mendapatkan distribusi topik. Ini menunjukkan topik mana saja yang paling dominan dalam dokumen tersebut beserta probabilitasnya.

Anda bisa mencoba mengubah num_topics dan melihat bagaimana kata-kata kunci dalam setiap topik berubah. Semakin banyak data yang Anda miliki, semakin jelas dan koheren topik-topik yang akan ditemukan oleh LDA.